In [ ]:
import math
import pandas as pd
import numpy as np
import plotly.express as px

# For rendering animation properly on colab 
# https://stackoverflow.com/revisions/57903750/4
import plotly.io as pio
#pio.renderers.default = "colab" 

class FretPositions(object):
  def __init__(self):
    self.setup()

  def setup(self, octave=1200, fourth=1200*22/53,cycles=28):
    self.octave = octave
    self.fourth = fourth
    self.cycles = cycles + 1
    return self

  def _generate(self):
    data1 = []

    for cycle in range(0, self.cycles):
      fourths = np.array([[x * self.fourth % self.octave,] for x in range(cycle)]).flatten().tolist()
      self.order = [i+1 for i,x in enumerate(fourths)]
      inverse = np.array([[(self.octave - x * self.fourth % self.octave),] for x in range(cycle)]).flatten().tolist()
      ratios = [math.exp(cents*math.log(2.0)/1200.0) for cents in fourths]
      inverse_ratios = [math.exp(cents*math.log(2.0)/1200.0) for cents in inverse]
      fret_fourths = [1 - 1/ratio for ratio in ratios]
      fret_fifths = [1 - 1/ratio for ratio in inverse_ratios]
      swara_order = ['s','m','n','g','d','r','M','N','G','D','R','P','s1','n1']
      swaras = ['s','s+',
                'r1','r2','R3','R4',
                'g1','g2','G3','G4',
                'm1','m2','M3','M4',
                'P-','P',
                'd1','d2','D3','D4',
                'n1','n2','N3','N4',
                'S-','S'
                ]
      self.tick_cents = [f'{x:.0f}' for i,x in enumerate(sorted(fourths + inverse))]
      self.tick_frets = [f'{x:.4f}' for i,x in enumerate(sorted(fret_fourths + fret_fifths))]
      #self.tick_cents = [f'{x:.0f} {swaras[i]}' for i,x in enumerate(sorted(fourths + inverse))] # Won't work. Must be number
      #self.tick_frets = [f'{x:.4f} {swaras[i]}' for i,x in enumerate(sorted(fret_fourths + fret_fifths))]

      data1.append(pd.DataFrame({
        'cycle' : cycle,
        'circle_of_fourths': fourths,
        'inverse' : inverse,
        'order': self.order,
        'labels' : ['m' for x in fourths],
        'inv_labels' : ['p' for x in fourths],
        'ratio' : ratios,
        'inverse_ratio' : inverse_ratios,
        'fourths_fret' : fret_fourths,
        'fifths_fret' : fret_fifths,
        }))
    self.data = data1
    self.source_animated = pd.concat(self.data)
    #print("Done _generate")
  def plot_cents(self):
    self._generate()
    fig = px.bar(self.source_animated, 
                 title="Cycle of fourths and fifths pitches in one octave VS the number of cycles",
                 x=['circle_of_fourths','inverse'], 
                 y='order', 
                 range_x=[0,self.octave], 
                 range_y=[0,self.cycles], 
                 #markers=True, 
                 animation_frame='cycle',
                 hover_data=["ratio"],

                 )
    fig.update_traces(width=2)
    fig.update_layout(transition_duration=3000)
    fig.update_xaxes(ticks="inside",tickvals=self.tick_cents, title="pitch in cents")
    fig.update_yaxes(ticks="inside", tickvals=[i for i in self.order ], title="number of cycles")
    fig.show()

  def plot_frets(self):
    fig = px.bar(self.source_animated,
                 title="Fret position from nut in one octave as a fraction of Vibrating String Length VS the number of cycles",
                 x=['fourths_fret','fifths_fret'], 
                 y='order', 
                 range_x=[0,0.5], 
                 range_y=[0,self.cycles], 
                 #markers=True, 
                 animation_frame='cycle',
                 hover_data=["ratio"]
                 )
    fig.update_traces(width=0.001)
    fig.update_layout(transition_duration=3000)
    fig.update_traces(marker_colorbar_tickfont_size=25, selector=dict(type='scatter'))
    fig.update_xaxes(ticks="inside",tickvals=self.tick_frets, title="fret distance from nut")
    fig.update_yaxes(ticks="inside", tickvals=[i for i in self.order ], title="number of cycles")
    fig.show()


## Each step in Cycle of fourths and its inverse relationship to octave

In [ ]:
#@title Pitch steps in Cycle of fourths and its inverse relationship to octave
fp = FretPositions()
fp.setup(cycles=13)
fp.plot_cents()


In [ ]:
#@title Fret position using Cycle of fourths and its inverse relationship to octave
fp.plot_frets()

In [ ]:
# This is not needed since plotly.renderers.default is set to colab
# from  https://stackoverflow.com/questions/47230817/plotly-notebook-mode-with-google-colaboratory
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext', 
            },
          });
        </script>
        '''))